In [14]:
import pandas as pd
from tqdm import tqdm
from database.strategy import Strategy
from database.market import Market
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import math
import numpy as np

In [15]:
strat_db = Strategy("unity")
market = Market()

In [16]:
market.connect()
sp5 = market.retrieve_data("sp500")
market.close()

In [17]:
strat_db.connect()
sim = strat_db.retrieve_data("sim")
strat_db.close()

In [18]:
availability = []
start = datetime(2015,1,1)
end = datetime(2020,9,1)
for ticker in list(sp5["Symbol"]):
    ticker = ticker.replace(".","-")
    td = sim[(sim["ticker"]==ticker) & (sim["date"]<=end)]
    try:
        availability.append({"ticker":ticker
                             ,"end_date":end <= td.iloc[len(td)-1]["date"]
                             ,"days":(td.iloc[len(td)-1]["date"]- td.iloc[0]["date"]).days
                            ,"records":len(td[td["date"] <= end])
                            ,"pct":math.ceil(len(td[td["date"] <= end])/(end - td.iloc[0]["date"]).days * 100)})
    except Exception as e:
        continue
stuff = pd.DataFrame(availability)
sp5_tickers = list(stuff[(stuff["pct"] >= stuff["pct"].mode().item() - 1) | (stuff["pct"] <= stuff["pct"].mode().item() + 1)]["ticker"].values)

In [19]:
sim["week"] = [x.week for x in sim["date"]]
sim["year"] = [x.year for x in sim["date"]]

In [20]:
wgs = sim.groupby(["year","week","ticker"]).mean().reset_index()

In [21]:
ticker_categories_weeklys = []
for ticker in wgs["ticker"].unique():
    ticker_wgs = wgs[wgs["ticker"] == ticker]
    ticker_wgs["actualclose"] = (ticker_wgs["adjclose"].shift(-1))
    ticker_wgs = ticker_wgs[:-1]
    ticker_wgs["actual_delta"] = (ticker_wgs["actualclose"] - ticker_wgs["adjclose"]) / ticker_wgs["adjclose"]
    ticker_wgs["sync"] = [0 if row[1]["actual_delta"] * row[1]["weekly_price_regression_prediction_delta"] >= 0 else 1 for row in ticker_wgs.iterrows()]
    ticker_wgs["category"] = ticker_wgs["sync"].rolling(window=5).mean()
    ticker_wgs = ticker_wgs[5:]
    ticker_wgs["category"] = [round(x) for x in ticker_wgs["category"]]
    ticker_categories_weeklys.append(ticker_wgs)
tcw = pd.concat(ticker_categories_weeklys)

In [22]:
final_sim = sim.merge(tcw[["year","week","ticker","category"]],on=["year","week","ticker"],how="left").dropna()
final_sim["predicted_delta"] = [-1 * row[1]["weekly_price_regression_prediction_delta"] \
                          if row[1]["category"] == 1 else row[1]["weekly_price_regression_prediction_delta"] \
                          for row in final_sim.iterrows()]

In [23]:
def backtest(sim,score,weekly_delta_req,ascending,seats):
    trades = []
    sim_columns = sim.columns
    sorted_column = "predicted_delta"
    sc = [x for x in sim_columns if "score" in x]
    rc = [x for x in sim_columns if "regression_prediction" in x]
    blacklist = pd.DataFrame([{"ticker":"ZZZZZ","start":datetime(2016,4,1),"end":datetime(2016,4,14)}])
    for i in tqdm(range(1,seats)):
        date = start
        while date <= end:
            if date.weekday() > 4:
                date = date + timedelta(days=7-date.weekday())
            blacklist_tickers = blacklist[(blacklist["start"] <= date) & (blacklist["end"] >= date)]["ticker"]
            taken = []
            for col in sc:
                sim = sim[sim[col] >= score]
            todays_sim = sim[(~sim["ticker"].isin(blacklist_tickers)) & (sim["ticker"].isin(sp5_tickers))
                            & (sim["date"] == date) & (sim["predicted_delta"] >= weekly_delta_req)]
            if todays_sim.index.size >= 1:
                offerings = todays_sim.sort_values(sorted_column,ascending=ascending)
                offerings = pd.concat([offerings[(offerings["category"]==0) \
                                                 & (offerings["trade_signal_classification_prediction"] == 1)], \
                                       offerings[(offerings["category"]==1) \
                                                 & (offerings["trade_signal_classification_prediction"] == 0)]])
                if offerings.index.size < 1:
                    date = date + timedelta(days=1)
                else:
                    for offering in range(offerings.index.size):
                        try:
                            trade = offerings.iloc[offering]
                            sell_date = trade["date"] + timedelta(days=1)
                            sell_trades = sim[(sim["date"] >= sell_date)  & (sim["date"] <= sell_date + timedelta(days=6)) & (sim["ticker"] == trade["ticker"])]
                            if sell_trades.index.size < 1:
                                if offering == offerings.index.size - 1:
                                    date = date + timedelta(days=1)
                                    break
                                else:
                                    continue
                            else:
                                sell_trades["delta"] = (sell_trades["adjclose"] - trade["adjclose"]) / trade["adjclose"]
                                sell_trade = sell_trades.sort_values("delta",ascending=False).iloc[0]
                                trade["sell_price"] = sell_trade["adjclose"]
                                trade["sell_date"] = sell_trade["date"]
                                trade["sell_delta"] = float(sell_trade["delta"])
                                trade["seat"] = i
                                trade["category"] = sell_trade["category"]
                                trade["predicted_delta"] = sell_trade["predicted_delta"]
                                blacklist = blacklist.append([{"ticker":trade["ticker"],"start":trade["date"],"end":sell_trade["date"]}])
                                trades.append(trade)
                                date = sell_trade["date"] + timedelta(days=1)
                                break
                        except Exception as e:
                            print("packaging",i,str(e))
                            date = date + timedelta(days=1)
            else:
                date = date + timedelta(days=1)
    return trades

In [24]:
epoch = 0
seats = 21
score_range = range(70,90,10)
wdr_range = range(0,15,5)
ascending_list = [True,False]
strat_db.connect()
strat_db.drop_table("tsmc_epochs")
for i in range(len(score_range)*len(wdr_range)*len(ascending_list)):
    strat_db.drop_table("tsmc_{}".format(i))
for s in tqdm(score_range):
    score = s/100
    for wdr in tqdm(wdr_range):
        for ascending in ascending_list:
            weekly_delta_req = wdr/100
            epoch_dict = {"epoch":epoch,"score":score,"ascending":ascending,"wdr":wdr}
            ts = backtest(final_sim.copy(),score,weekly_delta_req,ascending,seats)
            if len(ts) > 0:
                strat_db.store_data("tsmc_epochs",pd.DataFrame([epoch_dict]))
                strat_db.store_data("tsmc_{}".format(epoch),pd.DataFrame(ts))
            epoch += 1
            print(epoch,len(ts))
strat_db.close()

  0%|                                                                                                                                 | 0/3 [00:00<?, ?it/s]

  0%|                                                                                                                                | 0/20 [00:00<?, ?it/s]

  5%|█████▉                                                                                                                 | 1/20 [02:12<42:01, 132.73s/it]

 10%|███████████▉                                                                                                           | 2/20 [04:22<39:35, 131.96s/it]

 15%|█████████████████▊                                                                                                     | 3/20 [06:33<37:15, 131.53s/it]

 20%|███████████████████████▊                                                                                               | 4/20 [08:50<35:28, 133.06s/it]

 25%|█████████████████████████████▊                 

1 6548




  5%|█████▉                                                                                                                 | 1/20 [02:00<38:02, 120.13s/it]

 10%|███████████▉                                                                                                           | 2/20 [03:54<35:33, 118.52s/it]

 15%|█████████████████▊                                                                                                     | 3/20 [05:50<33:21, 117.76s/it]

 20%|███████████████████████▊                                                                                               | 4/20 [07:56<32:04, 120.26s/it]

 25%|█████████████████████████████▊                                                                                         | 5/20 [09:51<29:38, 118.57s/it]

 30%|███████████████████████████████████▋                                                                                   | 6/20 [11:55<28:02, 120.20s/it]

 35%|█████████████████████████████████████████▋   

2 6336




  5%|█████▉                                                                                                                 | 1/20 [02:10<41:17, 130.37s/it]

 10%|███████████▉                                                                                                           | 2/20 [04:17<38:49, 129.42s/it]

 15%|█████████████████▊                                                                                                     | 3/20 [06:29<36:53, 130.22s/it]

 20%|███████████████████████▊                                                                                               | 4/20 [08:42<34:55, 130.96s/it]

 25%|█████████████████████████████▊                                                                                         | 5/20 [10:49<32:28, 129.88s/it]

 30%|███████████████████████████████████▋                                                                                   | 6/20 [12:59<30:18, 129.91s/it]

 35%|█████████████████████████████████████████▋   

...

10 6594




  5%|██████                                                                                                                  | 1/20 [01:18<24:43, 78.07s/it]

 10%|████████████                                                                                                            | 2/20 [02:40<23:46, 79.26s/it]

 15%|██████████████████                                                                                                      | 3/20 [04:04<22:54, 80.86s/it]

 20%|████████████████████████                                                                                                | 4/20 [05:33<22:11, 83.20s/it]

 25%|██████████████████████████████                                                                                          | 5/20 [07:07<21:39, 86.63s/it]

 30%|████████████████████████████████████                                                                                    | 6/20 [08:44<20:55, 89.66s/it]

 35%|██████████████████████████████████████████   

11 4714




  5%|██████                                                                                                                  | 1/20 [01:18<24:56, 78.75s/it]

 10%|████████████                                                                                                            | 2/20 [02:42<24:05, 80.33s/it]

 15%|██████████████████                                                                                                      | 3/20 [04:10<23:24, 82.64s/it]

 20%|████████████████████████                                                                                                | 4/20 [05:43<22:51, 85.71s/it]

 25%|██████████████████████████████                                                                                          | 5/20 [07:19<22:13, 88.89s/it]

 30%|████████████████████████████████████                                                                                    | 6/20 [08:59<21:27, 91.96s/it]

 35%|██████████████████████████████████████████   

12 4751
